In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from scipy.integrate import solve_ivp

In [ ]:
def Θ(x, x_rev, λ, θ):
    xk, xj = np.meshgrid(x, x)
    return (xj - x_rev)/(1 + np.exp(-λ*(xk - θ)))

In [ ]:
def dΘ_dx(x, λ, θ):
    final = np.ones((x.size, x.size))*1/(1 + np.exp(-λ*(x - θ)))
    np.fill_diagonal(final, final.diagonal() + x*λ*np.exp(-λ*(x - θ))/(1+np.exp(-λ*(x - θ)))**2)
    return final

In [ ]:
def hr_dots(current, t0, b, i0, x_rev, λ, θ, μ, s, x_rest, α, n1, β, n2, G1, G2):
    x, y, z = map(lambda k: k.flatten(), np.split(current, 3))
    theta = Θ(x, x_rev, λ, θ)
    dots = np.zeros_like(current).reshape(3, -1)
    dots[0] = y - (x**3) + b*(x**2) + i0 - z - (α/n1)*np.sum(G1*theta, axis=1) - (β/n2)*np.sum(G2*theta, axis=1)
    dots[1] = 1 - 5*(x**2) - y
    dots[2] = μ*(s*(x - x_rest) - z)
    return np.hstack([dots[i] for i in range(3)])

In [ ]:
def jac(_, y_in):
    x, y, z = map(lambda k: k.flatten(), np.split(y_in, 3))
    dtheta_dx = dΘ_dx(x, λ, θ)
    dẋ_dx = -3*x**2 + 2*b*x - (α/n1)*G1*dtheta_dx - (β/n2)*G2*dtheta_dx
    dẋ_dy = np.ones_like(dẋ_dx)
    dẋ_dz = -np.ones_like(dẋ_dy)

    dẏ_dx = -10*x*np.ones_like(dẋ_dz)
    dẏ_dy = -np.ones_like(dẏ_dx)
    dẏ_dz = np.zeros_like(dẏ_dy)

    dż_dx = μ*s*np.ones_like(dẏ_dz)
    dż_dy = np.zeros_like(dż_dx)
    dż_dz = -μ*np.ones_like(dż_dy)

    j_x = [dẋ_dx, dẋ_dy, dẋ_dz]
    j_y = [dẏ_dx, dẏ_dy, dẏ_dz]
    j_z = [dż_dx, dż_dy, dż_dz]

    return np.vstack([np.hstack(j_x), np.hstack(j_y), np.hstack(j_z)])

In [ ]:
def cortex_size(mask, val):
    return int(np.sqrt(mask[mask == val].shape))

In [ ]:
n = np.loadtxt("connectomes/cat_connectome")/3
cortex_mask = np.zeros_like(n)
cortex_mask[:18, :18] = 1
cortex_mask[18:28, 18:28] = 2
cortex_mask[28:45, 28:45] = 3
cortex_mask[45:, 45:] = 4
G1 = n.copy()
G1[cortex_mask == 0] = 0
G2 = n.copy()
G2[cortex_mask != 0] = 0

In [ ]:
# For validation
for i in [1, 2, 3, 4]:
    print(i, cortex_size(cortex_mask, i))

In [ ]:
plt.matshow(n)
plt.colorbar()

In [ ]:
b = 3.2                           # Controls spiking frequency
i0 = 4.4*np.ones(n.shape[0])      # Input current ---- It's an array so we can add noise later
x_rev = 2                         # Reverse potential
λ = 10                            # Sigmoidal function parameter
θ = -0.25                         # Sigmoidal function parameter
μ = 0.01                          # Time scale of slow current
s = 4.0                           # Governs adaptation (whatever that means)
x_rest = 1.6                      # Resting potential
α = 0.210                         # Intra connection strength ---- VARIED PARAMETER
n1 = np.count_nonzero(G1, axis=1) # Number of intra connections from a given neuron
n1[n1 == 0] = 1                   # This is to remove a divide-by-zero; if n1 is 0, then so is G1
β = 0.040                         # Inter connection strength ---- VARIED PARAMETER
n2 = np.count_nonzero(G2, axis=1) # Number of inter connections from a given neuron
n2[n2 == 0] = 1                   # This is to remove a divide-by-zero; if n2 is 0, then so is G2

In [ ]:
ivs = np.zeros([3, n.shape[0]])   # Initial values [[x], [y], [z]]
ivs[0] = 4*np.random.random(n.shape[0]) - 2
ivs[1] = 0.2*np.random.random(n.shape[0])
ivs[2] = 0.2*np.random.random(n.shape[0])

In [ ]:
plt.plot(ivs[0], label="x")
plt.plot(ivs[1], label="y")
plt.plot(ivs[2], label="z")
plt.legend()

In [ ]:
plt.matshow(jac(0, ivs))
plt.colorbar()

In [ ]:
plt.hist(ivs[0], label="x")
plt.hist(ivs[1], label="y")
plt.hist(ivs[2], label="z")
plt.legend()

In [ ]:
params = (b, i0, x_rev, λ, θ, μ, s, x_rest, α, n1, β, n2, G1, G2)

In [ ]:
tmax = 4000
N = 100*tmax
t = np.linspace(0, tmax, N)

In [ ]:
def plot_beginning_and_end(y, start, end, p=0.99, legend=False, title=True):
    fig, [ax1, ax2] = plt.subplots(1, 2, sharey=True)
    for i in range(start, end):
        ax1.plot(y[:int(p*y.shape[0]), 0, i], label=i)
        ax2.plot(y[int((1 - p)*y.shape[0]):, 0, i], label=i)
    if legend:
        ax1.legend(loc="lower left")
    if title:
        plt.suptitle(f"First and last {100*p}% of neurons {start} - {end}")

In [ ]:
%%time
sol = solve_ivp(fun=lambda t_in, y_in: hr_dots(y_in, t_in, *params),
                t_span=(0, tmax), y0=ivs.reshape(ivs.size),
                max_step=1e-1, dense_output=True, method="LSODA", jac=jac)

In [ ]:
tmax/len(sol.t)

In [ ]:
vals = sol.sol(t).T

In [ ]:
vals = vals.reshape(-1, 3, 65)

In [ ]:
vals.shape

In [ ]:
plt.plot(range(18), vals[-1, 0, :18], "ro")
plt.plot(range(18, 28), vals[-1, 0, 18:28], "k^")
plt.plot(range(28, 45), vals[-1, 0, 28:45], "gX")
plt.plot(range(45, 65), vals[-1, 0, 45:], "bD")
plt.title("α = {}, β = {}, t = {}".format(α, β, tmax))

In [ ]:
p = 0.0025

In [ ]:
plot_beginning_and_end(vals, 0, 65, legend=False, p=p)

In [ ]:
plot_beginning_and_end(vals, 0, 18, p=p)

In [ ]:
plot_beginning_and_end(vals, 18, 28, p=p)

In [ ]:
plot_beginning_and_end(vals, 28, 45, p=p)

In [ ]:
plot_beginning_and_end(vals, 45, 65, p=p)

In [ ]:
α = 0.001
β = 0.001

In [ ]:
params = (b, i0, x_rev, λ, θ, μ, s, x_rest, α, n1, β, n2, G1, G2)

In [ ]:
%%time
sol = solve_ivp(fun=lambda t_in, y_in: hr_dots(y_in, t_in, *params),
                t_span=(0, tmax), y0=ivs.reshape(ivs.size),
                max_step=1e-1, dense_output=True, method="LSODA", jac=jac)

In [ ]:
tmax/len(sol.t)

In [ ]:
vals = sol.sol(t).T

In [ ]:
vals = vals.reshape(-1, 3, 65)

In [ ]:
vals.shape

In [ ]:
plt.plot(range(18), vals[-1, 0, :18], "ro")
plt.plot(range(18, 28), vals[-1, 0, 18:28], "k^")
plt.plot(range(28, 45), vals[-1, 0, 28:45], "gX")
plt.plot(range(45, 65), vals[-1, 0, 45:], "bD")
plt.title("α = {}, β = {}, t = {}".format(α, β, tmax))

In [ ]:
plot_beginning_and_end(vals, 0, 65, legend=False, p=p)

In [ ]:
plot_beginning_and_end(vals, 0, 18, p=p)

In [ ]:
plot_beginning_and_end(vals, 18, 28, p=p)

In [ ]:
plot_beginning_and_end(vals, 28, 45, p=p)

In [ ]:
plot_beginning_and_end(vals, 45, 65, p=p)